# Wasserstein GAN в задаче идентификации аномалий в геофизических данных
<br /><br />
В этом исследовании предлагается применить WGAN для идентификации аномалий в данных, описывающих состояние стратосферы на уровне 10гПа в северном полушарии.
<br /><br />
Известно, что обычно состояние стратосферы над северным полушарием в зимний период описывается как стратосферный полярный вихрь (СПВ). Это вихрь, образующийся под действием силы Кориолиса при условии выхолаживания полярной области стратосферы и, как следствие, возникновения градиента давления между полярной областью и умеренными широтами.

### Исходные данные
В этом исследовании состояние стратосферы будет описываться полями потенциальной завихренности (переменная `pv`) и высоты геопотенциала (переменная `gh`) на уровне 10гПа. Эти данные ограничены по широте: в исходных файлах представлены значения севернее $40^\circ$N, спроецированные с использованием полярной проекции. Кроме того, исходные данные уже отнормированы к интервалу $[0,1]$. Примеры полученного таким образом признакового описания состояний стратосферы приведены на рисунке ниже.
<br /><br />
Исходные данные исследования можно скачать по следующим ссылкам: [pv data](https://www.dropbox.com/s/ohwfyrmj4zl94q9/pv_data_projected_all.normed01.npy?dl=0); [gh data](https://www.dropbox.com/s/v3qjgzsls6et6cw/hgt_data_projected_all.normed01.npy?dl=0)
<br />
Маску для исключения несущественных зон снимка из рассмотрения можно скачать по ссылке: [mask data](https://www.dropbox.com/s/7s6lgdi01f8plkz/mask_256.npy?dl=0)

<center><img src="img/source_data_example.png" width=600></center>

### Внезапные стратосферные потепления как аномалии
Примерно с частотой один раз в 1.6 лет состояние стратосферы кардинально меняется, и сильный устойчивый вихрь, видный на диаграмме выше, распадается совсем или как минимум сильно возмущается и смещается с полюса. Эти состояния редки, и именно поэтому в этом исследовании будем считать их аномалиями. Цель исследования - применить набор нейросетевых моделей для поиска таких аномалий.

# Порождающие состязательные сети
Порождающие состязательные сети (ПСС, Generative Adversarial Networks, GAN) - нейросетевые модели, отличающиеся от обычных дискриминативных моделей в смысле методики обучения и сэмплирования данных.

<center><img src="img/GAN_en.png" width=600></center>

Терминология в GAN включает "реальные объекты" (real objects) - такие, которые порождены реальным процессом и даны в форме обучающей выборки, и "фейковые" или "порожденные" объекты (fake objects) - такие, которые являются результатом вычисления генератора $\mathcal{G}(\mathbb{\cdot})$ на входных данных $z$. При этом входные данные $z$ - это шум, порожденный из специального распределения. Обычно берут многомерное стандартное нормальное распределение. Порождаемые векторы шума (noise) также называют векторами скрытого представления (hidden representations или embeddings) по аналогии с терминологией автокодировщиков. В этом смысле генератор GAN - аналог декодера.

При этом дискриминатор $\mathcal{D}(\mathbb{\cdot})$ - подсеть, задача которой различать реальные и фейковые объекты, базируясь на их признаковом представлении. В определенном смысле дискриминатор - всего лишь нейросеть, решающая задачу бинарной классификации между реальными и фейковыми объектами.

Принцип обучения GAN состоит в том, что две подсети (генератор и дискриминатор) должны решать две противоположные задачи: генератор должен порождать примеры, настолько похожие на реальные, чтобы дискриминатор не мог их различить; при этом дискриминатор должен учиться все равно различать эти примеры.

Обозначения, которые вводятся для GAN:
- $z$ - шумовой вектор (noise) в пространстве скрытых представлений $\mathbb{Z}$;
- $x$ - признаковое описание реального объекта (!!! именно в пространстве признаков $\mathbb{X}$ !!!);
- $\mathcal{G}(z)$ - генератор, нейросеть, порождающая "фейковые" объекты из векторов $z$. Генератор переводит векторы пространства $\mathbb{Z}$ в векторы пространства $\mathbb{X}$:
$$
\mathcal{G}(\mathbf{\cdot}): \mathbb{Z}\to\mathbb{X}
$$
- $\mathcal{D}(x)$ - дискриминатор, нейросеть, решающая задачу классификации векторов пространства $\mathbb{X}$ на "реальные" и "фейковые".

Псевдоалгоритм обучения GAN:
<br /><br />
<center><img src="img/gan_algo.png" width=600></center>

Исходный алгоритм обучения GAN оказался нестабильным, и для его стабилизации были предложены усовершенствования, получившие название Wasserstein GAN ([WGAN](https://arxiv.org/abs/1701.07875)).
<br />
обозначения, принятые в этой статье:
- $g(z)$ - генератор;
- $f(x)$ - дискриминатор.
<br /><br />
Псевдоалгоритм обучения WGAN:
<br /><br />
<center><img src="img/wgan_algo.png" width=600></center>

В дальнейшем был предложен усовершенствованный алгоритм обучения WGAN с наложением ограничений на градиент дискриминатора $\mathcal{D}(x)$ по входным данным $x$ - Wasserstein GAN with Gradient Penalty ([WGAN-GP](https://arxiv.org/abs/1704.00028))
<br /><br />
Псевдоалгоритм обучения WGAN-GP:
<center><img src="img/wgan-gp-algo.png" width=600></center>

<hr>

В этом задании предлагается обратить внимание на свойство GAN аппроксимировать распределение данных. Это можно интуитивно понять на примере следующей пары диаграмм:
<center><img src="img/WGAN-mapping.png" width=600></center>

Здесь на левой панели условно отображены векторы скрытых представлений $z$ в пространстве скрытых представлений $\mathbb{Z}$. Напомним, что эти векторы порождаются из стандартного нормального распределения. На правой панели диаграммы условно отображаются объекты $x$ в пространстве признакового описания объектов $\mathbb{X}$. Здаесь они отображены в двумерном пространстве, однако нужно помнить, что в предлагаемой задаче это векторы в пространстве размерности `2*256*256`, поскольку признаковое описание состояний стратосферы составлено двумя полями (`pv` и `gh`), каждое из которых представляет собой матрицу размером `256x256`.

На диаграмме условно отображено соответствие областей в пространствах $\mathbb{Z}$ и $\mathbb{X}$, в которых сосредоточена основная часть объектов выборки (точки оранжевого цвета). Точками синего цвета условно отображены объекты, являющиеся выбросами (аномалиями) для данных выборки. С точки зрения распределения переменной $z$ эти объекты лежат за пределами основной массы распределения, то есть, могут быть интерпретированы, например, как векторы, норма которых больше перцентиля уровня $98\%$. В это же время, в пространстве признакового описания $\mathbb{X}$ соответствующие объекты, вычисляемые как $\mathcal{G}(z)$ могут считаться аномалиями (синие точки на правой панели диаграммы).

С точки зрения задачи идентификации аномалий в этом случае остается только один вопрос: **как в пространстве $\mathbb{X}$ построить разделяющую поверхность**, соответствующую разделяющей поверхности, проходящей по перцентилю $98\%$ нормы векторов $z$ в пространстве $\mathbb{Z}$. Это при условии, что **вряд ли найдется возможность обратить преобразование генератора и получить $\mathcal{G}^{-1}(x)$**. А именно такое преобразование понадобилось бы для того, чтобы протестировать вновь поступающие на анализ события $x\in\mathbb{X}$ на принадлежность области "обычных" или "аномальных" объектов в пространстве $\mathbb{Z}$.

#### Применение WGAN-GP для порождения сбалансированной выборки "обычных" и "аномальных" объектов

В качестве альтернативного решения такой задачи в этом задании (во второй части) предлагается создать нейросетевой классификатор $\mathcal{F}(x)$, способный разделять объекты $x$ на обычные, характерные для выборки, и аномальные. Вопрос в том, - как его обучить?

В случае, когда в нашем распоряжении есть генератор $\mathcal{G}(z)$, обученный порождать как "обычные" объекты, так и "аномальные", можно сгенерировать достаточное количество обучающих примеров для такого бинарного классификатора $\mathcal{F}(x)$. При этом метка $y_i$ "обычный"/"аномальный" будет доступна по условию порождения таких примеров: в силах исследователя в момент генерации шумового вектора $z_i$ классифицировать его, руководствуясь его нормой:

- Если $|z_i|_2>q_{0.98}(|z|_2)$, то объект может считаться аномальным, $y_i=1$
- Иначе объект может считаться обычным, $y_i=0$

Отметим также, что в силах исследователя регулировать порождение сбалансированной выборки, что важно для обучения надежного классификатора. Для этого на этапе порождения векторов $z$ следует следить за сбалансированностью их с точки зрения классификации на "обычные" и "аномальные".

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm
from libs.service_defs import *
import torch.autograd as autograd
from scipy import stats
from torch.utils.tensorboard import SummaryWriter

from typing import Tuple, List, Type, Dict, Any

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Проверим доступность GPU
torch.cuda.device_count()

In [ ]:
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [ ]:
class DS(Dataset):
    def __init__(self,
                 pv_fname: str = ''
                 gh_fname: str = '',
                 mask_fname: str = '',
                 transform: Any = None):
        
        '''
        здесь должен быть код инициализациия экземпляра класса DS
        В нем следует считать данные и записать в виде атрибутов этого экземпляра класса
        Также следует сохранить в качестве атрибута этого экземпляра класса преобразование(я) transform,
            которые должны будут применяться к данным
        
        '''
        ################################
        ###    YOUR CODE HERE        ###
        ################################
        

    def __getitem__(self, index):
        x = np.zeros((1,2,256,256))
        '''
        Здесь должно быть описание процедуры составления (возможно, из заранее прочитанных данных)
            признакового описания запрашиваемого по индексу index объекта,
            а также применения преобразований transform, если они есть.
        Этот метод должен выдавать признаковое описание объекта.
        Подсказка: размерность признакового описания одного объекта должна быть [1,2,256,256], где
        1 - нумерует (единственный) объект в этой выборке
        2 - нумерует "каналы" данных (pv и gh)
        256,256 - индексирует пространственные переменные
        '''
        ################################
        ###    YOUR CODE HERE        ###
        ################################

        return x
    

    def __len__(self):
        length = 0
        '''
        Этот метод должен возвращать полное количество объектов в выборке
        '''
        ################################
        ###    YOUR CODE HERE        ###
        ################################

        return length

In [ ]:
train_transforms = lambda x: Tensor(x)

In [ ]:
train_dataset = DS('/path/to/pv_data_projected_all.normed01.npy',
                   '/path/to/gh_data_projected_all.normed01.npy',
                   '/path/to/mask_256.npy',
                   train_transforms)

Для контроля можно отобразить порождаемые этим классом объекты:

In [ ]:
indices = np.random.randint(0, len(train_dataset), size=8)
samples = [train_dataset[idx] for idx in indices]

fig = plt.figure(figsize=(2, 8), dpi=300)
for idx,sample in enumerate(samples):
    sample_np = sample.numpy()
    p = plt.subplot(8,2,idx*2+1)
    plt.imshow(sample_np[0], cmap='gray')
    plt.axis('off')
    
    p = plt.subplot(8,2,idx*2+2)
    plt.imshow(sample_np[1], cmap='gray')
    plt.axis('off')
    
plt.tight_layout()
fig.patch.set_facecolor('white')

Далее следует описать подсети генератора $\mathcal{G}(\cdot)$ и дискриминатора $\mathcal{D}(\cdot)$

Напомним, генератор принимает на вход вектор размерности `n_inputs` (в случае мини-батча размером `N` - матрицу `N x n_inputs`) и возвращает тензор размером `N x 2 x 256 x 256`

При этом дискриминатор принимает на вход тензоры размером `N x 2 x 256 x 256` и действует как бинарный классификатор: выходное значение - одно действительное число на каждый объект. То есть, размерность выходного тензора должна быть `N x 1`. В случае WGAN-GP это выходное значение не ограничивается по величине и может принимать все действительные значения (подумайте, какая в этом случае должна быть функция активации выходного слоя).

In [ ]:
class Generator(nn.Module):
    
    def __init__(self, n_inputs=128):
        super(Generator, self).__init__()
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################

    def forward(self, x_noise):
        x = x_noise
        ################################
        ###    YOUR CODE HERE        ###
        ################################
    
        return x

In [ ]:
class Discriminator(nn.Module):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################

    def forward(self, x):
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################
        
        return x

Далее применим описанные классы генератора и дискриминатора для создания WGAN-GP:

In [ ]:
latent_dim = 128

In [ ]:
gen = Generator(n_inputs=latent_dim)
gen = gen.to(DEVICE)

In [ ]:
dsc = Discriminator()
dsc = dsc.to(DEVICE)

In [ ]:
# Также следует создать оптимизаторы отдельно для генератора и для дискриминатора
opt_gen = None # YOUR CODE HERE
opt_dsc = None # YOUR CODE HERE

Для модели WGAN-GP применяется регуляризация на градиент выхода дискриминатора по его входным данным. Здесь предлагается воспользовать его реализацией, приведенной ниже:

In [ ]:
def compute_gradient_penalty(model_dsc, real_samples, fake_samples):
    """
    Calculates the gradient penalty loss for WGAN GP
    model_dsc - дискриминатор
    real_samples - признаковое описание реальных примеров
    fake_samples - признаковое описание примеров, порожденных генератором.
    """
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = model_dsc(interpolates)
    fake = Variable(Tensor(d_interpolates.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(outputs=d_interpolates,
                              inputs=interpolates,
                              grad_outputs=fake,
                              create_graph=True,
                              retain_graph=True,
                              only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
def train_model(model_gen: torch.nn.Module, 
                model_dsc: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                optimizer_gen: torch.optim.Optimizer,
                optimizer_dsc: torch.optim.Optimizer,
                batch_size = 32,
                max_epochs = 10,
                n_critic = 5,
                lambda_gp = 10):
    
    '''
    В этой функции следует запрограммировать обучение и валидацию WGAN-GP.
    
    '''
    
    train_loader = None # YOUR CODE HERE
    
    lr_scheduler_gen = None # YOUR CODE HERE
    lr_scheduler_dsc = None # YOUR CODE HERE
        
    # Полезно будет записать эволюцию функций потерь по ходу обучения - ее можно будет потом отобразить на диаграмме
    val_loss_history = []
    gen_loss_history = []
    dsc_loss_history = []
    lgen = 0.0
    ldsc = 0.0
    
    # Далее следует написать цикл оптимизации генератора и дискриминатора,
    # руководствуясь псевдоалгоритмом, приведенным в описании выше
    # Замечание: Не забудьте применить маску как к реальным данным (это можно сделать в генераторе данных DS),
    #           так и к данным, порождаемым генератором!
    
       
    
    for epoch in range(max_epochs):
        print(f'Starting epoch {epoch+1} of {max_epochs}')
        
        model_gen.train()
        model_dsc.train()
        
        with tqdm(total=len(train_loader)) as pbar:
            for idx, real_batch in enumerate(train_loader):
                
                ################################
                ###    YOUR CODE HERE        ###
                ################################
                # шаг оптимизации дискриминатора

                if idx % n_critic == 0:
                    
                    ################################
                    ###    YOUR CODE HERE        ###
                    ################################
                    # шаг оптимизации генератора
                
                pbar.update(1)
                
                pbar.set_postfix({'step': idx+1, 'loss_GEN': lgen, 'loss_DSC': ldsc})
                
        # Тут рекомендуется сохранять модели каждую эпоху
        ################################
        ###    YOUR CODE HERE        ###
        ################################
        
        # Валидация - оценка функции потерь WGAN-GP (без регуляризационного члена) как D(x_real) - D(x_fake)
        
        model_gen.eval()
        model_dsc.eval()
        val_loss_epoch = 0.0
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################
        
        val_loss_history.append(val_loss_epoch)
        
        lr_scheduler_gen.step()
        lr_scheduler_dsc.step()
        
    return val_loss_history, dsc_loss_history, gen_loss_history

In [ ]:
loss_history, dsc_loss_history, gen_loss_history = train_model(gen, dsc,
                                                               train_dataset,
                                                               opt_gen, opt_dsc,
                                                               max_epochs=300,
                                                               n_critic=4,
                                                               lambda_gp=10)

Для оценки качества обучения, полезно посмотреть на кривые обучения:

In [ ]:
f = plt.figure(figsize=(8,3), dpi=300)
p = plt.subplot(1,3,1)
_ = plt.scatter(np.arange(len(loss_history)), loss_history, s=1)

p = plt.subplot(1,3,2)
_ = plt.scatter(np.arange(len(dsc_loss_history)), dsc_loss_history, s=1)

p = plt.subplot(1,3,3)
_ = plt.scatter(np.arange(len(gen_loss_history)), gen_loss_history, s=1)

Далее предлагается применить генератор для порождения новых примеров и оценить их правдоподобность.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)

In [ ]:
for idx, real_batch in enumerate(train_loader):
    break

In [ ]:
# При необходимости можно загрузить нужное состояние генератора
gen = Generator(n_inputs=latent_dim)
gen.load_state_dict(torch.load('./path/to/model/checkpoint/gen.pt'))
gen = gen.to(DEVICE)

In [ ]:
real_batch = real_batch.to(DEVICE)
noise = Variable(torch.tensor(np.random.normal(0, 1, (len(real_batch), latent_dim)),
                              dtype=torch.float, device=DEVICE))
fake_batch = gen(noise)

# Замечание: Не забудьте применить маску к данным, порождаемым генератором!
################################
###    YOUR CODE HERE        ###
################################

In [ ]:
real_batch = real_batch.detach().cpu().numpy()
fake_batch = fake_batch.detach().cpu().numpy()

In [ ]:
f = plt.figure(figsize=(4,4), dpi=300)
p = plt.subplot(2,2,1)
plt.imshow(real_batch[0,0,...], cmap='gray')
plt.axis('off')
p = plt.subplot(2,2,2)
plt.imshow(real_batch[0,1,...], cmap='gray')
plt.axis('off')

p = plt.subplot(2,2,3)
plt.imshow(fake_batch[2,0,...], cmap='gray')
plt.axis('off')
p = plt.subplot(2,2,4)
plt.imshow(fake_batch[2,1,...], cmap='gray')
plt.axis('off')

In [ ]:
indices = np.random.choice(np.arange(32), 16, replace=False)

In [ ]:
f = plt.figure(figsize=(4,4), dpi=300)

for i,idx in enumerate(indices):
    p = plt.subplot(4,4,i+1)
    plt.imshow(fake_batch[idx,0,...], cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

In [ ]:
f = plt.figure(figsize=(4,4), dpi=300)

for i,idx in enumerate(indices):
    p = plt.subplot(4,4,i+1)
    plt.imshow(fake_batch[idx,1,...], cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

### Использование обученного генератора $\mathcal{G}(\mathbb{\cdot})$

В этой части задания предлагается использовать генератор, обученный в первой части, для порождения примеров, похожих на представленные в обучающей выборке. Напомним, что векторы $z$ порождаются многомерным стандартным нормальным распределением, однако в этом задании следует изменить соотношение обычных экземпляров и экземпляров-аномалий. Этого можно добиться двумя способами:

- Сэмплировать с отклонением (отвержением) примеров. В этом случае векторы $z$ порождаются из стандартного нормального распределения, однако при этом экземпляры с нормой менее перцентиля уровня $98\%$ отклоняются с вероятностью, близкой к $98\%$ (вычислите, какова в точности должна быть вероятность отвержения, если Вы выбрали этот способ сэмплирования). Этот способ гарантирует получение сбалансированной выборки "обычных" и "аномальных" векторов $z$, однако не гарантирует равномерного сэмплирования внутри области "обычных" примеров;
- Сэмплировать равномерно в некоторой области, ограниченной нормой $z$: $z\leq z_{max}$. Здесь тоже должно быть задействовано сэмплирование с отвержением, однако условие отвержения будет другим. Если это ваш выбор, вычислите, каково должно быть $z_{max}$, чтобы выборка обычных и аномальных примеров была сбалансирована. Напомним: известно, что разделение обычных и аномальных примеров в пространстве $\mathbb{Z}$ проходит по значению нормы вектора $|z|_2=q_{0.98}(|z|_2)$. Обратите внимание, что эта величина зависит от размерности $n$ случайной величины $z$, поэтому правильнее писать $q_{0.98}(|z|_2, n)$.

**Подсказка**: поскольку $z$ распределена нормально с единичной матрицей ковариаций $\Sigma$, то каждая из компонент этого вектора распределена нормально с дисперсией $\sigma^2=1$; это означает, что $(|z|_2)^2=\sum_{j=1}^{n}z_j^2$ имеет распределение хи-квадрат, а $|z|_2=\sqrt{\sum_{j=1}^{n}z_j^2}$, соответственно, имеет распределение хи. Для вычисления величины $q_{0.98}(|z|_2, n)$ можно воспользоваться методами распределения, реализованными в библиотеке `scipy`: [`scipy.stats.chi`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi.html). 

После того, как мы знаем, в пределах каких значений $|z|_2$ лежат векторы $z$, отображающиеся в "обычные примеры", нужно научиться сэмплировать их равномерно. Для реализации равномерного сэмплирования в $n$-шаре можно воспользоваться функцией равномерного сэмплирования в многомерном $n$-кубе [numpy.random.rand](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html#numpy.random.rand) с отвержением экземпляров по норме $|z|_2$

После этого следует научиться сэмплировать векторы за пределами этого $n$-шара радиусом $q_{0.98}(|z|_2, n)$. Например, можно сэмплировать равномерно из шара радиусом $R_{outer}=2^{1/n}$ с отвержением всех экземпляров внутри вложенного шара, содержащего "обычные" примеры.

**Подсказка**: можно сразу сэмплировать из шара радиусом $R_{outer}=q_{0.98}(|z|_2, n)*2^{1/n}$ и делить примеры на две категории: обычные (для которых $|z|_2\leq q_{0.98}(|z|_2, n)$) и аномальные (все остальные экземпляры).

### Об эффективном сэмплировании из $n$-мерного шара радиусом $R$

В тот момент, когда требуется порождать векторы, равномерно распределенные в $n$-мерном шаре радиуса $R$, в первую очередь на ум приходит равномерное сэмплирование из $n$-мерного куба со стороной $2R$ с отклонением примеров за пределами шара. Однако с возрастанием количества измерений **$n$** эффективность (скорость) алгоритма такого равномерного сэмплирования резко падает. Вместо этого лучше воспользоваться другим способом:

#### Псевдоалгоритм 1 - равномерного сэмплирования из $n$-мерного шара радиусом $R$
1. Породить $\mathbf{s}\in \mathbb{R}^n$ равномерно на поверхности $n-1$-мерной единичной сферы.
2. Породить $c\sim U[0,1]$ (из равномерного распределения с поддержкой $[0,1]$)
3. Вернуть вектор $\mathbf{b}=s*c^{1/n}*R$

При этом нужно еще уметь равномерно порождать векторы $\mathbf{s}\in \mathbb{R}^n$ на поверхности $n-1$-мерной единичной сферы. Для этого можно воспользоваться следующим алгоритмом:

#### Псевдоалгоритм 2 - равномерного сэмплирования из $n-1$-мерной единичной сферы
1. Породить $n$-мерный вектор, распределенный нормально с диагональной единичной матрицей ковариаций и нулевым вектором средних: $\mathbf{d}\sim\mathcal{N}(\mathbf{0}, \mathbb{I})$.
2. Вычислить $L_2$-норму вектора $\mathbf{d}$:
$$
|\mathbf{d}|_2=\sqrt{\sum_{i=1}^{n}d_i^2}
$$
3. Вернуть вектор-направление единичной длины $\tilde{\mathbf{d}}=\frac{\mathbf{d}}{|\mathbf{d}|_2}$

In [ ]:
def SamplingUnitSphereUniform(ndim=2, nsamples=100):
    '''
    Эта функция должна порождать примеры, равномерно распределенные по единичной n-мерной сфере
        (см. псевдоалгоритм 2)
    '''
    ################################
    ###    YOUR CODE HERE        ###
    ################################
    return d

In [ ]:
def SamplingBallUniform(ndim=2, radius=1, nsamples=100):
    '''
    Эта функция должна порождать примеры, равномерно распределенные в единичном n-мерном шаре
        (см. псевдоалгоритм 1)
    '''
    ################################
    ###    YOUR CODE HERE        ###
    ################################
    return b

In [ ]:
def SampleInnerAndOuterExamples(ndim=2, radius_inner=1, radius_outer=2, nsamples=100):
    '''
    В этой функции следует порождать примеры, равномерно распределенные в ndim-мерном шаре диаметром radius_outer
        и делить их на samples_inner ("обычные") и samples_outer ("аномальные")
        по норме векторов с пороговым значением radius_inner
    '''
    ################################
    ###    YOUR CODE HERE        ###
    ################################
    return samples_inner, samples_outer

### Проверьте свою реализацию сэмплирования

В качестве проверки можно породить, скажем, 10000 примеров, равномерно распределенных в двумерном шаре (на круге, $n=2$). Внешний радиус нужно поставить в значение $R_{outer}=q_{0.98}(|z|_2, n)*2^{1/n}$, внутренний - в значение $R_{inner}=q_{0.98}(|z|_2, n)$

Ожидаемый результат:
- примеров `examples_inner` и `examples_outer` должно получиться примерно одинаковое количество;
- примеры распределены равномерно по кругу радиусом примерно 4;
- "обычные" (внутри круга радиусом $R_{inner}$) и "аномальные" (в кольце между окружностями радиусом $R_{inner}$ и $R_{outer}$) примеры визуально не перемешиваются;

In [ ]:
examples_inner, examples_outer = SampleInnerAndOuterExamples(ndim=2,
                                                             radius_inner=stats.chi.ppf(0.98, 2),
                                                             radius_outer=stats.chi.ppf(0.98, 2)*np.sqrt(2),
                                                             nsamples=10000)
examples_inner.shape[0], examples_outer.shape[0]

In [ ]:
f = plt.figure(figsize=(4,4), dpi=300)
plt.scatter(examples_inner[:,0], examples_inner[:,1], s=1, color='orange')
plt.scatter(examples_outer[:,0], examples_outer[:,1], s=1, color='blue')
_ = plt.axis('equal')

### Еще раз проверьте свою реализацию сэмплирования

В качестве второй проверки можно породить, скажем, 10000 примеров, равномерно распределенных в шаре высокой размерности (например, $n=128$). Внешний радиус нужно поставить в значение $R_{outer}=q_{0.98}(|z|_2, n)*2^{1/n}$, внутренний - в значение $R_{inner}=q_{0.98}(|z|_2, n)$

Ожидаемый результат:
- примеров `examples_inner` и `examples_outer` должно получиться примерно одинаковое количество;

In [ ]:
chi_p98 = 0.0 # YOUR CODE HERE - значение перцентиля уровня 98% распределения хи с количеством степеней свободы latent_dim
print(chi_p98)

In [ ]:
examples_inner, examples_outer = SampleInnerAndOuterExamples(ndim=latent_dim,
                                                             radius_inner=chi_p98,
                                                             radius_outer=chi_p98*np.power(2, 1/latent_dim),
                                                             nsamples=10000)
print(examples_inner.shape[0], examples_outer.shape[0])

## Обучение классификатора $\mathcal{F}(\mathbf{\cdot})$

Итак, к этому моменту у вас должно быть все готово для обучения нейросети $\mathcal{F}(\mathbf{x})$, которая будет разделять "обычные" и "аномальные" примеры. Более конкретно, у вас должны быть готовы:
- обученный генератор - нейросетевая модель, преобразующая векторы $z\in\mathbb{Z}$ в примеры $x$ в пространстве признаков $\mathbb{X}$:
$$
\mathcal{G}(\cdot): \mathbb{Z}\to\mathbb{X}
$$
- механизм равномерного порождения "обычных" и "аномальных" векторов $z\in\mathbb{Z}$. Для этих векторов заранее известны метки, являются ли они обычными ($y=0$) или аномальными ($y=1$)

**ПРИМЕЧАНИЕ**
Генератор на этом этапе обучать не нужно! Не забудьте переключить его в режим исполнения!

In [ ]:
# При необходимости можно загрузить нужное состояние генератора
gen = Generator(n_inputs=latent_dim)
gen.load_state_dict(torch.load('./path/to/model/checkpoint/gen.pt'))
gen = gen.to(DEVICE)

Далее нужно описать классификатор. Это нейросеть, выполняющая задачу бинарной классификации. Скорее всего, вам захочется сделать ее свёрточной.

ПРИМЕЧАНИЕ: не забудьте, что здесь уже будет использоваться постановка задачи бинарной классификации. Подумайте, какими в этом случае должны быть:
- размерность выходного слоя классификатора;
- функция активации выходного слоя;
- функция потерь.

In [ ]:
class Classifier(nn.Module):
    
    def __init__(self):
        super(Classifier, self).__init__()
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################

    def forward(self, x):
        
        ################################
        ###    YOUR CODE HERE        ###
        ################################
        
        return x

In [ ]:
clf = Classifier()
clf = clf.to(DEVICE)

In [ ]:
def train_classifier(model_gen: torch.nn.Module, 
                     model_clf: torch.nn.Module,
                     batch_size = 64,
                     batches_per_epoch = 128,
                     max_epochs = 512):
    
    '''
    В этой функции следует описать цикл оптимизации нейросети-классификатора.
    Не забудьте, что примеры для этой нейросети вы порождаете генератором, обученным выше.
    При этом шумовые векторы z для генератора вы порождаете, используя функции сбалансированного сэмплирования,
        описанные выше.
        
    Не забывайте и здесь применять маску к данным, порождаемым генератором!
    '''
    
    ################################
    ###    YOUR CODE HERE        ###
    ################################
    
    val_loss_history = []
    model_gen.eval()
    
    for epoch in range(max_epochs):
        print(f'Starting epoch {epoch+1} of {max_epochs}')
        
        model_clf.train()
        
        ####################################
        ###    YOUR CODE HERE            ###
        ### classifier optimization loop ###
        ####################################
                
        
        model_clf.eval()
        val_loss_epoch = 0.0
        
        ####################################
        ###    YOUR CODE HERE            ###
        ###  classifier evaluation loop  ###
        ####################################
            
        val_loss_history.append(val_loss_epoch)
        print('Eval loss on epoch %d: %f' % (epoch+1, loss_epoch))
        
        lr_scheduler.step()
        
    return val_loss_history

In [ ]:
val_loss_history = train_classifier(model_gen=gen,
                                    model_clf=clf,
                                    batch_size = 128,
                                    batches_per_epoch = 128,
                                    max_epochs = 512)

## Применение классификатора $\mathcal{F}(\mathbf{\cdot})$

Вы проделали большую работу! Теперь можно применить обученный классификатор для идентификации аномалий в данных.

Для этого нужно применить ее ко всем данным выборки.

- создайте новый экземпляр класса `DS`;
- создайте `dataloader` на основе этих данных. При этом не перемешивайте его: сейчас важен порядок обработки и ответов классификатора;
- в цикле примените классификатор $\mathcal{F}(\mathbf{\cdot})$ ко всем данным; сохраните ответы классификатора в массив;
- запишите индексы элементов выборки, которые классифицированы как аномальные, в массив `idx_anomalies`;
- воспользуйтесь заготовкой кода ниже для отображения объектов, которые классифицированы как аномальные и обычные.

In [ ]:
ds_test = DS('/path/to/pv_data_projected_all.normed01.npy',
             '/path/to/gh_data_projected_all.normed01.npy',
             '/path/to/mask_256.npy',
             train_transforms)

In [ ]:
test_loader = torch.utils.data.DataLoader(ds_test, shuffle=False, batch_size=64)

In [ ]:
clf = Classifier()
clf.load_state_dict(torch.load('/path/to/classifier/checkpoint/file.pt'))
clf = clf.to(DEVICE)

## Анализ результатов.

Примените обученный классификатор ко всем данным изначальной выборки.

Проанализируйте результаты:
- каково распределение вероятностей примеров быть аномальными, оцениваемое этим классификатором. Отобразите в виде гистограммы.
- Выберите пороговое значение для этих оценок вероятностей. Если применить разделение по этому пороговому значению, - каково количество примеров, отнесенных к обычным и к аномальным?
- Отобразите несколько состояний стратосферы, отнесенных к обычным, и несколько состояний, отнесенных к аномальным.